In [ ]:
def compute_energy(partitions, image):
    reshaped_image = image.reshape(-1, 3)
    unflatten_indexes, edges = get_best_colors(reshaped_image, n_colors=params["n_colors"], bins=params["bins"])
    main_colors = get_main_colors(unflatten_indexes, np.array(edges))
    distributions = get_distributions(main_colors, image)
    
    energies = [0] * len(partitions)
    for n in tqdm(range(reshaped_image.shape[0])):
        pixel = reshaped_image[n]
        data_costs = data_cost(pixel, distributions)
        neighboors = get_neighboors(n, image.shape[:-1])
        for index, partition in enumerate(partitions):
            label = get_label(n, partition)
            energies[index] += data_costs[label]
            for i, j in neighboors:
                label_neighboor = get_label(np.ravel_multi_index((i, j), image.shape[:-1]), partition)
                cost = smooth_cost(label, label_neighboor, params["epsilon"])
                energies[index] += cost
    return energies